In [40]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.utils.data_utils import get_file
import h5py
import numpy as np
execfile('midiutils.py')

mod_layer = 2
l1_size = 70*mod_layer
l2_size = 60*mod_layer
l6_size = 70*mod_layer
maxlen = 16*8
num_tracks = 6

song = '../daft_punk-robot_rock.mid'
# dynamic creation of ranges, and selection of tracks
ranges, tracks = midi_analyze(song)
ranges = ranges[0:num_tracks]
notes = 2*sum([b-a+1 for a, b in ranges])

model = Sequential()

model.add(LSTM(l1_size, return_sequences=True, input_shape=( maxlen, notes )))
model.add(Dropout(0.2))

model.add(LSTM(l2_size, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(l6_size, return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense( notes ))
model.add(Activation('softmax'))

model.load_weights('../../f0b36dca58ujeztr1gi7.h5')
model.compile(loss='categorical_crossentropy', optimizer='adamax')

Exception: Layer weight shape (212, 140) not compatible with provided weight shape (156, 140)

In [14]:
def sample(a, ranges, temperature = 1.0):
    # helper function to sample an index from a probability array
    sa = np.log(a) / temperature
    #sa = np.exp(sa) / np.sum(np.exp(sa))

    # sampling experiment
    lowerNotes, upperNotes, span = md.split_tracks(ranges)
    notes = sum(span)
    offs = [0 for _ in range(len(lowerNotes)+1)]
    ind = []
    for i in range(len(span)) :
        offs[i+1] = offs[i] + span[i]
    b1 = np.array([]) # notes
    b2 = np.array([]) # sustain
    for i in range(len(offs)-1) :
        pd1 = np.exp(sa[offs[i]:offs[i+1]])/(np.sum(np.exp(sa[offs[i]:offs[i+1]]))*10)
        pd2 = np.exp(sa[notes+offs[i]:notes+offs[i+1]])/(np.sum(np.exp(sa[notes+offs[i]:notes+offs[i+1]])))
        #pd1 = sa[offs[i]:offs[i+1]]
        #pd2 = sa[notes+offs[i]:notes+offs[i+1]]
        b1 = np.hstack((b1, np.random.binomial([1.]*len(pd1), pd1)))
        b2 = np.hstack((b2, np.random.binomial([1.]*len(pd2), pd2)))
    return np.hstack((b1,b2))

In [15]:
def generate_song(model, ranges, maxlen, path, seed):
    print('Generate ====================')
    notes = 2*sum([b-a+1 for a, b in ranges])
    #seed = np.array(np.random.randint(2, size=(1, maxlen, notes)), dtype=bool)
    for temperature in [ 0.5, 0.7, 1 ]:
        for _ in range(maxlen):
            preds = model.predict(seed, verbose=0)[0]
            preds = sample(preds, ranges, temperature)
            seed = np.array([np.vstack((seed[0][1:,:],preds))])
    
        onm = md.output_to_nm(seed[0])
        s = string.lowercase+string.digits # name string options
        song_name = ''.join(random.sample(s,20))

        md.output_midi_poly_array(onm, ranges, path+'songs/'+song_name+'.mid')
    return

In [37]:
maxlen = 16*8	# time decimation
#sc = 100	# batch length

lowerNotes, upperNotes, span = split_tracks(ranges)
l = sum(span)

x_in = nm_to_input(training_set)

sc = int(np.floor(x_in.shape[0]/(maxlen+1)))
x_in = x_in[0:sc*(maxlen+1),:]

for _ in range(8):
    x_in = np.vstack((x_in, x_in))

x_in = np.reshape(x_in, (256*sc, maxlen+1, 2*l))
x = x_in[:,:-1,:]
y = x_in[:,-1,:]
out_ranges = ranges

path = '.'
print(2*l)
print(notes)
#generate_song(model, out_ranges, maxlen, path, np.array([x[0,:,:]]))

212
156
